In [ ]:
!pip install ultralytics pdf2image pillow opencv-python pyyaml tqdm
!apt-get install poppler-utils

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 35.1 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 41 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.12 [186 kB]
Fetched 186 kB in 1s (371 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 121703 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.12_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.12) ...
Setting up poppler-utils (22.02.0-2ubuntu0.12) ...
Processing triggers for man-db (2.10.2-1) ...


In [ ]:
import os

os.makedirs("/content/data/images", exist_ok=True)
os.makedirs("/content/data/labels", exist_ok=True)
os.makedirs("/content/predictions", exist_ok=True)

print("Folders created.")

Folders created.


In [ ]:
from pdf2image import convert_from_path
import os
import re

pdf_dir = "/content/pdf"
out_dir = "/content/data/images"

def safe_name(s):
    return re.sub(r'[\\/:\*\?"<>\|]', '_', s)

pdf_files = [f for f in os.listdir(pdf_dir) if f.lower().endswith(".pdf")]
print("PDF count:", len(pdf_files))

for pdf in pdf_files:
    pdf_path = os.path.join(pdf_dir, pdf)
    safe_pdf = safe_name(pdf)
    pages = convert_from_path(pdf_path, dpi=200)

    for i, page in enumerate(pages, 1):
        out_path = f"{out_dir}/{safe_pdf}_page_{i}.png"
        page.save(out_path, "PNG")
        print("Saved:", out_path)


PDF count: 45
Saved: /content/data/images/отр-22.pdf_page_1.png
Saved: /content/data/images/отр-22.pdf_page_2.png
Saved: /content/data/images/отр-22.pdf_page_3.png
Saved: /content/data/images/разрешназемл-2.pdf_page_1.png
Saved: /content/data/images/перечень-.pdf_page_1.png
Saved: /content/data/images/перечень-.pdf_page_2.png
Saved: /content/data/images/Перечень-2.pdf_page_1.png
Saved: /content/data/images/Перечень-2.pdf_page_2.png
Saved: /content/data/images/локалсметчб-.pdf_page_1.png
Saved: /content/data/images/локалсметчб-.pdf_page_2.png
Saved: /content/data/images/локалсметчб-.pdf_page_3.png
Saved: /content/data/images/ТЗ-41-чб.pdf_page_1.png
Saved: /content/data/images/ТЗ-41-чб.pdf_page_2.png
Saved: /content/data/images/ТЗ-41-чб.pdf_page_3.png
Saved: /content/data/images/отр-41-.pdf_page_1.png
Saved: /content/data/images/отр-41-.pdf_page_2.png
Saved: /content/data/images/Письмо-41-.pdf_page_1.png
Saved: /content/data/images/Письмо-41-.pdf_page_2.png
Saved: /content/data/images/До

In [ ]:
import json
import re
import os

json_path = "/content/jsons/selected_annotations.json"
images_dir = "/content/data/images"
labels_dir = "/content/data/labels"

CATEGORY_TO_ID = {"signature": 0, "stamp": 1, "qr": 2}

def safe_basename(s: str):
    return re.sub(r'[\\/:\*\?"<>\|]', '_', s)

with open(json_path, "r", encoding="utf-8") as f:
    data = json.load(f)

for pdf_name, pages in data.items():
    safe_pdf = safe_basename(pdf_name)

    for page_key, page_val in pages.items():
        page_num = page_key.split("_")[-1]
        pw = page_val["page_size"]["width"]
        ph = page_val["page_size"]["height"]

        image_name = f"{safe_pdf}_page_{page_num}.png"
        image_path = f"{images_dir}/{image_name}"
        label_path = f"{labels_dir}/{image_name.replace('.png', '.txt')}"

        lines = []

        for ann in page_val["annotations"]:
            for _, info in ann.items():
                cat = info["category"]
                if cat not in CATEGORY_TO_ID:
                    continue
                cid = CATEGORY_TO_ID[cat]
                x = info["bbox"]["x"]
                y = info["bbox"]["y"]
                w = info["bbox"]["width"]
                h = info["bbox"]["height"]

                xc = (x + w/2) / pw
                yc = (y + h/2) / ph
                wn = w / pw
                hn = h / ph

                lines.append(f"{cid} {xc} {yc} {wn} {hn}")

        with open(label_path, "w") as f:
            f.write("\n".join(lines))

        print("Label saved:", label_path)


Label saved: /content/data/labels/локалсмета-.pdf_page_3.txt
Label saved: /content/data/labels/Разрешназемлю-41-.pdf_page_1.txt
Label saved: /content/data/labels/письмо-11.pdf_page_2.txt
Label saved: /content/data/labels/дефект-41-чб.pdf_page_2.txt
Label saved: /content/data/labels/отр-1.pdf_page_1.txt
Label saved: /content/data/labels/Археология-31-.pdf_page_2.txt
Label saved: /content/data/labels/локалсмета-32-.pdf_page_3.txt
Label saved: /content/data/labels/Письмо-42.pdf_page_1.txt
Label saved: /content/data/labels/ТЗ-.pdf_page_1.txt
Label saved: /content/data/labels/ТЗ-.pdf_page_3.txt
Label saved: /content/data/labels/отр-32-.pdf_page_1.txt
Label saved: /content/data/labels/отр-32-.pdf_page_3.txt
Label saved: /content/data/labels/Археология-2.pdf_page_2.txt
Label saved: /content/data/labels/перечень-.pdf_page_1.txt
Label saved: /content/data/labels/перечень-.pdf_page_2.txt
Label saved: /content/data/labels/отр-41-.pdf_page_1.txt
Label saved: /content/data/labels/отр-11.pdf_page_1.

In [ ]:
import yaml

yaml_dict = {
    "path": "/content/data",
    "train": "/content/data/images",
    "val": "/content/data/images",
    "names": ["signature", "stamp", "qr"],
    "nc": 3
}

with open("/content/data.yaml", "w") as f:
    yaml.dump(yaml_dict, f)

print(open("/content/data.yaml").read())


names:
- signature
- stamp
- qr
nc: 3
path: /content/data
train: /content/data/images
val: /content/data/images



In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")
model.train(
    data="/content/data.yaml",
    epochs=40,
    imgsz=1024,
    batch=8,
    name="digital_inspector",
)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.228 🚀 Python-3.12.12 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=40, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1024, int8=False, iou=0.7, keras=False, ko

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7bf766260080>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04

In [ ]:
from ultralytics import YOLO
import cv2
import json
import os

model_path = "/content/runs/detect/digital_inspector/weights/best.pt"
model = YOLO(model_path)

results_json = {}

image_files = [f for f in os.listdir("/content/data/images") if f.endswith(".png")]

for img_name in image_files:
    img_path = f"/content/data/images/{img_name}"
    preds = model(img_path, conf=0.25)[0]

    image = cv2.imread(img_path)
    objects = []

    if preds.boxes is not None:
        for box in preds.boxes:
            x1, y1, x2, y2 = box.xyxy[0].tolist()
            cls = int(box.cls[0])
            conf = float(box.conf[0])

            objects.append({
                "category": ["signature", "stamp", "qr"][cls],
                "bbox": [float(x1), float(y1), float(x2-x1), float(y2-y1)],
                "confidence": conf
            })

            cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (0,255,0), 2)

    cv2.imwrite(f"/content/predictions/{img_name}", image)
    results_json[img_name] = {"objects": objects}

with open("/content/predictions/final_output.json", "w", encoding="utf-8") as f:
    json.dump(results_json, f, indent=2, ensure_ascii=False)

print("/content/predictions/final_output.json")


image 1/1 /content/data/images/АПЗ-2.pdf_page_3.png: 1024x736 1 qr, 350.1ms
Speed: 14.8ms preprocess, 350.1ms inference, 7.5ms postprocess per image at shape (1, 3, 1024, 736)

image 1/1 /content/data/images/АПЗ-31-.pdf_page_1.png: 1024x736 1 qr, 336.0ms
Speed: 8.4ms preprocess, 336.0ms inference, 1.1ms postprocess per image at shape (1, 3, 1024, 736)

image 1/1 /content/data/images/АПЗ-41-чб.pdf_page_11.png: 1024x736 1 qr, 328.5ms
Speed: 7.9ms preprocess, 328.5ms inference, 1.0ms postprocess per image at shape (1, 3, 1024, 736)

image 1/1 /content/data/images/дефект-31-.pdf_page_2.png: 1024x736 6 signatures, 1 stamp, 323.9ms
Speed: 8.0ms preprocess, 323.9ms inference, 1.1ms postprocess per image at shape (1, 3, 1024, 736)

image 1/1 /content/data/images/отр-31-.pdf_page_1.png: 1024x736 5 signatures, 1 stamp, 327.5ms
Speed: 8.2ms preprocess, 327.5ms inference, 1.0ms postprocess per image at shape (1, 3, 1024, 736)

image 1/1 /content/data/images/ТЗ-.pdf_page_1.png: 1024x736 1 signatur

In [ ]:
from google.colab import files
files.download("/content/predictions/final_output.json")

In [ ]:
from ultralytics import YOLO
import cv2
import json
import os

os.makedirs("/content/predictions/pages", exist_ok=True)

model_path = "/content/runs/detect/digital_inspector/weights/best.pt"
model = YOLO(model_path)

ID_TO_CAT = {0: "signature", 1: "stamp", 2: "qr"}

results_json = {}

image_dir = "/content/data/images"
out_dir = "/content/predictions/pages"

image_files = [f for f in os.listdir(image_dir) if f.endswith(".png")]

for img_name in image_files:
    img_path = f"{image_dir}/{img_name}"

    pred = model(img_path, conf=0.25)[0]

    img = cv2.imread(img_path)

    objects = []

    if pred.boxes is not None:
        for box in pred.boxes:
            x1, y1, x2, y2 = box.xyxy[0].tolist()
            cls = int(box.cls[0])
            conf = float(box.conf[0])

            objects.append({
                "category": ID_TO_CAT.get(cls, str(cls)),
                "bbox": [float(x1), float(y1), float(x2 - x1), float(y2 - y1)],
                "confidence": conf
            })

            cv2.rectangle(
                img,
                (int(x1), int(y1)),
                (int(x2), int(y2)),
                (0, 255, 0),
                3
            )
            cv2.putText(
                img,
                f"{ID_TO_CAT.get(cls)} {conf:.2f}",
                (int(x1), int(y1) - 10),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.6,
                (0, 255, 0),
                2
            )

    out_path = f"{out_dir}/{img_name}"
    cv2.imwrite(out_path, img)

    print("Saved annotated:", out_path)

    results_json[img_name] = objects

with open("/content/predictions/final_pages_output.json", "w", encoding="utf-8") as f:
    json.dump(results_json, f, indent=2, ensure_ascii=False)

print("/content/predictions/pages/")
print("/content/predictions/final_pages_output.json")


image 1/1 /content/data/images/АПЗ-2.pdf_page_3.png: 1024x736 1 qr, 331.5ms
Speed: 8.3ms preprocess, 331.5ms inference, 1.0ms postprocess per image at shape (1, 3, 1024, 736)
Saved annotated: /content/predictions/pages/АПЗ-2.pdf_page_3.png

image 1/1 /content/data/images/АПЗ-31-.pdf_page_1.png: 1024x736 1 qr, 328.8ms
Speed: 8.1ms preprocess, 328.8ms inference, 1.1ms postprocess per image at shape (1, 3, 1024, 736)
Saved annotated: /content/predictions/pages/АПЗ-31-.pdf_page_1.png

image 1/1 /content/data/images/АПЗ-41-чб.pdf_page_11.png: 1024x736 1 qr, 325.8ms
Speed: 8.0ms preprocess, 325.8ms inference, 1.1ms postprocess per image at shape (1, 3, 1024, 736)
Saved annotated: /content/predictions/pages/АПЗ-41-чб.pdf_page_11.png

image 1/1 /content/data/images/дефект-31-.pdf_page_2.png: 1024x736 6 signatures, 1 stamp, 340.7ms
Speed: 8.0ms preprocess, 340.7ms inference, 1.1ms postprocess per image at shape (1, 3, 1024, 736)
Saved annotated: /content/predictions/pages/дефект-31-.pdf_page_2